In [35]:
import json
import os
import wikipediaapi

In [17]:
page_of_all_composers = "Liste der vom NS-Regime oder seinen Verbündeten verfolgten Komponisten"

In [27]:
wiki_wiki = wikipediaapi.Wikipedia(language="de",extract_format=wikipediaapi.ExtractFormat.WIKI)
wiki_page = wiki_wiki.page(page_of_all_composers)

In [28]:
def print_categories(page):
    categories = page.categories
    for title in sorted(categories.keys()):
        print("%s: %s" % (title, categories[title]))

In [29]:
print_categories(wiki_page)

Kategorie:Liste (Komponisten): Kategorie:Liste (Komponisten) (id: ??, ns: 14)
Kategorie:Liste (NS-Opfer): Kategorie:Liste (NS-Opfer) (id: ??, ns: 14)


In [30]:
persecuted_composers = [key for key in wiki_page.links.keys()]
persecuted_composers

['Abel Ehrlich',
 'Admiral Horthy',
 'Alexander von Zemlinsky',
 'Alexandre Tansman',
 'Alfons Josef Biron',
 'Alfred Kastner (Musiker)',
 'Alfred Rosé',
 'Alfred Szendrei',
 'Andre Asriel',
 'Andries de Rosa',
 'Antal Doráti',
 'Anton Webern',
 'Armin Berg',
 'Arno Nadel',
 'Arnold Maria Walter',
 'Arnold Schönberg',
 'Arthur Chitz',
 'Arthur Kleiner',
 'Arthur Löwenstein',
 'Arthur Rebner',
 'Arthur Willner',
 'Artur Schnabel',
 'Ashley Vernon',
 'Bernard Grün',
 'Bernd Bergel',
 'Bernhard Paumgartner',
 'Bernhard Sekles',
 'Bert Silving',
 'Berthold Goldschmidt',
 'Bertus van Lier',
 'Bob Hanf',
 'Bohuslav Martinů',
 'Bronislau Kaper',
 'Bruno Granichstaedten',
 'Bruno Walter',
 'Béla Bartók',
 'Carlo Sigmund Taube',
 'Charlotte Schlesinger',
 'Daniël Belinfante',
 'Darius Milhaud',
 'David Beigelman',
 'Dick Kattenburg',
 'Dol Dauber',
 'Drittes Reich',
 'Edmund Eysler',
 'Eduard Steuermann',
 'Efim Schachmeister',
 'Egon Ledec',
 'Egon Lustgarten',
 'Egon Wellesz',
 'Einsatzstab R

In [42]:
file = "data/komponisten_texte_deutsch.json"
# check if file exists
if os.path.isfile(file):
    print("Texts have already been scraped to:", file)
    with open(file, "r", encoding="utf8") as json_file:
        json_file_text = json_file.read()
        composer_texts_dict = json.loads(json_file_text)

else:
    # get german text from each composers wikipedia page and save in json-file
    with open(file, "w") as json_file:
        composer_texts_dict = {}
        for composer in persecuted_composers:
            composer_wiki_page = wiki_wiki.page(composer)
            # exclude non-existing articles, and all articles which are not about a composer
            valid_composer = False
            for key in composer_wiki_page.categories.keys():
                if "Komponist" in key or "komponist" in key:
                    valid_composer = True
                    break
            if valid_composer:
                print("Processing wikipedia page of", composer)
                # composer_wiki_page.sections TODO: Überschriften, Literatur etc. aus Text entfernen?
                composer_text = composer_wiki_page.text
                # composer_text = composer_text.replace("\n\n\n", " ")
                # composer_text = composer_text.replace("\n\n", " ")
                # composer_text = composer_text.replace("\n", " ")
                composer_texts_dict[composer] = composer_text
                # get other languages:
                # composer_wiki_page.langlinks
        json.dump(composer_texts_dict, json_file, indent=4, ensure_ascii=False)

Texts have already been scraped to: data/komponisten_texte_deutsch.json


In [43]:
print(len(composer_texts_dict), "of", len(persecuted_composers), "are valid wikipedia pages of composers.")

213 of 313 are valid wikipedia pages of composers.
